In [5]:
import json, math
import matplotlib.pyplot as plt

In [6]:
syms = []
symfile = open('../snp500sym.csv','r')
symlines = symfile.readlines()
for line in symlines:
    syms.append(line.strip())
print(syms[:5])
print(f'no of stocks = {len(syms)}')

['MMM', 'AOS', 'ABT', 'ABBV', 'ABMD']
no of stocks = 505


In [7]:
bal = open('INCOME_STATEMENT/AAL.json', 'r')
inc = open('BALANCE_SHEET/AAL.json', 'r')
bal = json.load(bal)
inc = json.load(inc)
inc = inc['annualReports'][0]
bal = bal['annualReports'][0]
income_statement_indicators = [u[0] for u in inc.items()]
balance_sheet_indicators = [u[0] for u in bal.items()]
income_statement_indicators = set(income_statement_indicators)
balance_sheet_indicators = set(balance_sheet_indicators)
# print(income_statement_indicators)
# print(balance_sheet_indicators)
indicators = income_statement_indicators.union(balance_sheet_indicators)
print(indicators)
# print(len(indicators))


{'shortTermDebt', 'currentNetReceivables', 'incomeTaxExpense', 'incomeBeforeTax', 'netIncome', 'goodwill', 'longTermInvestments', 'otherCurrentLiabilities', 'totalCurrentAssets', 'researchAndDevelopment', 'totalRevenue', 'totalNonCurrentLiabilities', 'depreciation', 'totalNonCurrentAssets', 'treasuryStock', 'costOfRevenue', 'intangibleAssetsExcludingGoodwill', 'grossProfit', 'comprehensiveIncomeNetOfTax', 'fiscalDateEnding', 'investments', 'totalShareholderEquity', 'shortTermInvestments', 'deferredRevenue', 'retainedEarnings', 'propertyPlantEquipment', 'operatingExpenses', 'depreciationAndAmortization', 'capitalLeaseObligations', 'totalCurrentLiabilities', 'commonStockSharesOutstanding', 'interestAndDebtExpense', 'accumulatedDepreciationAmortizationPPE', 'intangibleAssets', 'ebitda', 'currentAccountsPayable', 'cashAndShortTermInvestments', 'longTermDebtNoncurrent', 'otherCurrentAssets', 'currentLongTermDebt', 'otherNonCurrentLiabilities', 'otherNonCurrrentAssets', 'costofGoodsAndServic

In [8]:
period_file = open(f'../data/QuarterlyData/2021_Q4.csv', 'r')
indicators = period_file.readline()
lines = period_file.readlines()
ticker = []
for line in lines:
    line = line.strip().split(',')
    ticker.append(line[0])
indicators = indicators.strip().split(',')
# print(indicators)
indicatos_ind = dict()
for i in range(len(indicators)):
    indicatos_ind[indicators[i].lower()] = i+1
print(indicatos_ind)

{'ticker': 1, 'reportedcurrency': 2, 'totalassets': 3, 'totalcurrentassets': 4, 'cashandcashequivalentsatcarryingvalue': 5, 'cashandshortterminvestments': 6, 'inventory': 7, 'currentnetreceivables': 8, 'totalnoncurrentassets': 9, 'propertyplantequipment': 10, 'accumulateddepreciationamortizationppe': 11, 'intangibleassets': 12, 'intangibleassetsexcludinggoodwill': 13, 'goodwill': 14, 'investments': 15, 'longterminvestments': 16, 'shortterminvestments': 17, 'othercurrentassets': 18, 'othernoncurrrentassets': 19, 'totalliabilities': 20, 'totalcurrentliabilities': 21, 'currentaccountspayable': 22, 'deferredrevenue': 23, 'currentdebt': 24, 'shorttermdebt': 25, 'totalnoncurrentliabilities': 26, 'capitalleaseobligations': 27, 'longtermdebt': 28, 'currentlongtermdebt': 29, 'longtermdebtnoncurrent': 30, 'shortlongtermdebttotal': 31, 'othercurrentliabilities': 32, 'othernoncurrentliabilities': 33, 'totalshareholderequity': 34, 'treasurystock': 35, 'retainedearnings': 36, 'commonstock': 37, 'com

In [9]:
ratios = [
    # ['pe', [], []], 
    # ['eps', [], []],
    # ['de', ['totalLiabilities'], ['totalShareHolderEquity']], 
    ['da', ['totalLiabilities'], ['totalAssets']],
    # ['roe', ['netincome'], ['totalShareHolderEquity']],
    ['roa', ['netincome'], ['totalAssets']],
    # ['net_profit_margin', ['netincome'], ['totalrevenue']],
    # ['current_ratio', ['totalCurrentAssets'], ['totalCurrentLiabilities']],
    # ['quick_ratio', ['totalcurrentassets', '-inventory'], ['totalcurrentliabilities']],
    ['at', ['totalrevenue'], ['totalAssets']],
    # ['debt_ratio', ['totalliabilities'], ['totalassets']],
]

In [10]:
for year in range(2017, 2022):
    for qtr in range(1, 5):
        period = f'{year}_Q{qtr}'
        period_file = open(f'../data/QuarterlyData/{period}.csv', 'r')
        header = period_file.readline()
        lines = period_file.readlines()
        info = []
        for line in lines:
            line = line.strip()
            line = line.split(',')
            info.append(line)
        # print(info[:5])
        period_ratios = open(f'../data/QuarterlyRatios/{period}.csv', 'w')
        ratio_header = ['ticker']
        for ratio in ratios:
            ratio_header.append(ratio[0])
        period_ratios.write(','.join(ratio_header)  + '\n')
        for i in range(len(ticker)):
            to_write = [ticker[i]]
            for ratio in ratios:
                
                num = 0
                den = 0
                    
                try:
                    for nums in ratio[1]:
                        if (nums.lower())[0] == '-':
                            num -= float(info[i][indicatos_ind[(nums.lower())[1:]]])
                        else:
                            num += float(info[i][indicatos_ind[nums.lower()]])
                    for dens in ratio[2]:
                        if (dens.lower())[0] == '-':
                            den -= float(info[i][indicatos_ind[(dens.lower())[1:]]])
                        else:
                            den += float(info[i][indicatos_ind[dens.lower()]])
                    to_write.append(str(num/den))
                    pass
                except:
                    if num != 0:
                        # print(num, den)
                        pass
                    to_write.append('#####')
                    pass
            
            period_ratios.write(','.join(to_write) + '\n')

In [11]:
tickerset = set(ticker)
badtickerset = set()
year_start = 2017
year_end = 2020
for year in range(year_start, year_end):
    for qtr in range(1, 5):
        period = f'{year}_Q{qtr}'
        period_file = open(f'../data/QuarterlyRatios/{period}.csv', 'r')
        header = period_file.readline()
        lines = period_file.readlines()
        period_file.close()
        cool = 0
        for line in lines:
            ok = True
            line = line.strip().split(',')
            for word in line:
                if word == '#####':
                    ok = False
                    break
            if not ok:
                if tickerset.__contains__(line[0]):
                    tickerset.remove(line[0])
                badtickerset.add(line[0])
            else:
                cool += 1
        print(cool,end=' ')
print()
tickerset.remove("TWTR")
print(len(tickerset), tickerset)
print(len(badtickerset), badtickerset)

471 476 479 471 478 482 483 488 490 489 490 490 
416 {'LOW', 'MTB', 'BRO', 'K', 'UAA', 'CNC', 'OTIS', 'FTV', 'ADSK', 'MAA', 'ADM', 'FBHS', 'CHTR', 'ADP', 'XOM', 'TGT', 'ABT', 'MU', 'PNR', 'ATVI', 'ILMN', 'J', 'MMM', 'ANTM', 'IPG', 'NVR', 'CL', 'MAR', 'ITW', 'PPG', 'FDS', 'JPM', 'ROST', 'UNH', 'VLO', 'SRE', 'AES', 'ICE', 'ROP', 'KMX', 'MCHP', 'MDT', 'ECL', 'AMD', 'NDSN', 'GE', 'GM', 'SPG', 'DTE', 'EIX', 'AAPL', 'SO', 'JBHT', 'AIG', 'MO', 'ETN', 'WMT', 'AMP', 'PEG', 'PFG', 'APH', 'VRSK', 'EOG', 'MAS', 'ETR', 'IFF', 'ALB', 'CHD', 'TSLA', 'NI', 'TEL', 'NCLH', 'CAH', 'NWS', 'AXP', 'HAL', 'BXP', 'CRL', 'HES', 'BK', 'TFC', 'AEE', 'NFLX', 'IEX', 'IQV', 'GILD', 'SNA', 'VNO', 'VRSN', 'HBAN', 'NWSA', 'BIIB', 'APA', 'HON', 'ANSS', 'PNC', 'MKTX', 'KLAC', 'MTD', 'RMD', 'PSX', 'RE', 'UHS', 'ZBRA', 'BSX', 'DVA', 'EXPE', 'WEC', 'CCI', 'WAT', 'ROL', 'NWL', 'CPRT', 'DFS', 'NUE', 'FAST', 'MA', 'PEAK', 'IP', 'KR', 'CRM', 'DLR', 'GNRC', 'NLSN', 'MDLZ', 'PAYX', 'MCK', 'AKAM', 'CHRW', 'FRT', 'RJF', 'CLX', 'EB

In [12]:
for year in range(year_start, year_end):
    for qtr in range(1, 5):
        period = f'{year}_Q{qtr}'
        period_file = open(f'../data/QuarterlyRatios/{period}.csv', 'r')
        header = period_file.readline()
        lines = period_file.readlines()
        period_file.close()
        period_file = open(f'../data/QuarterlyRatiosClean/{period}.csv', 'w')
        period_file.write(header)
        for line in lines:
            line = line.strip().split(',')
            if tickerset.__contains__(line[0]):
                period_file.write(line[0] + ',' + ','.join(line[1:]) + '\n')

In [13]:
for year in range(year_start, year_end):
    for qtr in range(1, 5):
        period = f'{year}_Q{qtr}'
        period_file = open(f'../data/QuarterlyRatiosClean/{period}.csv', 'r')
        header = period_file.readline()
        lines = period_file.readlines()
        period_file.close()
        period_file = open(f'../data/QuarterlyRatiosCleanNormalised/{period}.csv', 'w')
        period_file.write(header)
        avg = [0 for _ in range(len(lines[0].strip().split(','))-1)]
        stddev = [0 for _ in range(len(lines[0].strip().split(','))-1)]
        for line in lines:
            line = line.strip().split(',')
            for i in range(len(line)-1):
                avg[i] += float(line[i+1])
        for i in range(len(avg)):
            avg[i] /= len(lines)
        for line in lines:
            line = line.strip().split(',')
            for i in range(len(line)-1):
                stddev[i] += (float(line[i+1]) - avg[i])**2
        for i in range(len(stddev)):
            stddev[i] = math.sqrt(stddev[i] / len(lines))
        
        for line in lines:
            line = line.strip().split(',')
            to_write = [line[0]]
            for i in range(len(line)-1):
                # append if data is within 3 std deviations
                if abs(float(line[i+1]) - avg[i]) < 3 * stddev[i]:
                    to_write.append(str(float((float(line[i+1]) - avg[i])/stddev[i])))
            if len(to_write) == len(line):
                period_file.write(','.join(to_write) + '\n')